# Lab 4: Collecting Data from Leuschner

In [ ]:
# ssh -XY radiolab@leuschner.berkeley.edu -p 31
# password = Cud4b4ck
# cd anoising

In [1]:
import ugradio
import numpy as np
import astropy
import pyfits
from ugradio import nch
from astropy.io import fits
from ugradio import agilent
from ugradio import leo
from ugradio import leusch
import leuschner

## Telescope Controls

In [ ]:
tele = ugradio.leusch.LeuschTelescope() # interface for controlling the Leuschner Telescope
tele.get_pointing()  # returns the current (alt,az) of the dish in degrees
tele.point(alt,az) # moves the telescope to a specified alt, az in degrees
tele.main() # moves the telescope to maintenance position

## Noise

In [ ]:
LeuschNoise = ugradio.leusch.LeuschNoise() # interface to the noise diode
LeuschNoise.on()
LeuschNoise.off()

## Local Oscillator (LO)

In [ ]:
lo = ugradio.agilent.SynthDirect() # interface to the LO
lo.get_frequency()
lo.get_amplitude()
lo.set_frequency(633, 'MHz')
lo.set_amplitude(10, 'dBm')

synth = ugradio.agilent.SynthClient(host='127.0.0.1') # idk what this does

## Spectrometer Controls

In [2]:
spec = leuschner.Spectrometer('10.0.1.2') # interface to the spectrometer at the specified IP address
spec.check_connected() # checks that you are connected and ready to take data
spec.int_time() # returns how long each spectrum is integrated for
spec.read_spec('name.fits', N, (ra, dec), 'eq') # reads N spectra and stores output in a fits file, writing ra/dec coordinates into the header of the file. Note that this does NOT point the telescope.
# e.g. spec.read_spec('data_apr20.fits', 15,(50, 60),'eq')

## For Loops to Point Telescope and Collect Data

Note: need to instantiate interfaces tele and spec before running

In [ ]:
l = np.arange(l1, l2, 2)
b = np.arange(b1, b2, 2)

ra_dec = []

for i in range(len(l)):
    for j in range(len(b)):
        galactic = SkyCoord(l[i], b[j], frame='galactic', unit='deg')
        icrs = galactic.transform_to('icrs')
        ra = icrs.ra.degree
        dec = icrs.dec.degree
        ra_dec.append([ra, dec])

for i in range(len(ra_dec)):
    ra = ra_dec[i][0]
    dec = ra_dec[i][1]
    jd = ugradio.timing.julian_date()
    alt, az = ugradio.coord.get_altaz(ra, dec, jd=jd, lat=leo.lat, lon=leo.lon, alt=leo.alt, equinox='J2019')
    tele.point(alt, az)
    spec.read_spec('data_date_'+str(ra)+'_'+str(dec)+'.fits', 15, (ra, dec), 'eq') # reads 15 spectra